<h1 align=center><font size = 4> Nearby_Seattle_food_choice_exploration </font></h1>

##### Hello dear viewer,
##### I have added screenshot of the visuals in the report of the project.
##### Also some of the key points which were in question referred in (https://github.com/lalitsomnathe/Coursera_Capstone/blob/master/Nearby_Seattle_food_choice_exploration_lalitsomnathe_Intro.txt) project document are answered and highlighted at very end considering all four metro divisions.

#### Importing libraries

In [1]:
import urllib
import requests
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

#### Extracting the Lattitude and longitudes by web scrapping as mentioned in the data extraction step of the project document. 
#### https://github.com/lalitsomnathe/Coursera_Capstone/blob/master/Nearby_Seattle_food_choice_exploration_lalitsomnathe_data.txt
I am using beautifulsoup and some extra line code for extracting the lat,lng.
And also creating a fuction so that it can be used for all four metro divisions.

In [2]:
#'http://worldpopulationreview.com/us-cities/seattle-population/'

def get_lat_lng(url):
    r=requests.get(url)
    data=r.text

    soup = BeautifulSoup(data)

    soup1=str(soup)

    var1=soup1.find('www.google.com/maps/?q=')
    #print(var1)
    var_len=len('www.google.com/maps/?q=')
    #print(var_len)

    var2=soup1[var1+var_len:].find("\"")
    #print(var2)

    latlang=soup1[var1+var_len:var1+var_len+var2]
    #print(latlang)

    lat,lang=latlang.split(",")

    lat,lang=round(float(lat),4),round(float(lang),4)
    #print(lat,lang)
    label=url.split("/")[-2].split("-")[0]
    return [lat,lang,label]

Testing with seattle data below :-

In [3]:
seattle_lat_lang=get_lat_lng('http://worldpopulationreview.com/us-cities/seattle-population/')
print(seattle_lat_lang)
print(type(seattle_lat_lang))

[47.6062, -122.3321, 'seattle']
<class 'list'>


Using functions for extracting lat,lng data for all metro divisions.:-

In [4]:
bellevue_lat_lang=get_lat_lng('http://worldpopulationreview.com/us-cities/bellevue-population/')
tacoma_lat_lang=get_lat_lng('http://worldpopulationreview.com/us-cities/tacoma-population/')
everett_lat_lang=get_lat_lng('http://worldpopulationreview.com/us-cities/everett-wa-population/')

print (seattle_lat_lang)
print (bellevue_lat_lang)
print (tacoma_lat_lang)
print (everett_lat_lang)

[47.6062, -122.3321, 'seattle']
[47.6104, -122.2007, 'bellevue']
[47.2529, -122.4443, 'tacoma']
[47.979, -122.2021, 'everett']


#### Importing libraries for data analysis and data visualization.

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    ------------------------------------------------------------
                         

#### Creating a list with all metro divisions, this will be helpful while using in folium map :-

In [6]:
LAT_LANG=[seattle_lat_lang, bellevue_lat_lang, tacoma_lat_lang, everett_lat_lang]
for i in LAT_LANG:
    print(i[0])

47.6062
47.6104
47.2529
47.979


#### Using Folium map for seattle location and also inserting locations for other metro divisions.

In [7]:
# create map of Manhattan using latitude and longitude values
map_seattle = folium.Map(location=[seattle_lat_lang[0], seattle_lat_lang[1]], zoom_start=9)

# add markers to map
# for lat, lng, label in zip(seattle_lat_lang[0], seattle_lat_lang[1], seattle_lat_lang[2]):
for i in LAT_LANG:
    lat,lng,label=i
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_seattle)  
    
map_seattle

#### Foursquare API credentials :-

In [8]:
# CLIENT_ID = 'your-client-ID' # your Foursquare ID
# CLIENT_SECRET = 'your-client-secret' # your Foursquare Secret

CLIENT_ID = 'WI10VXATDOABDVBGEVP5NS2ONF5SRWCTIJXRFP44XWHUY1MD' # your Foursquare ID
CLIENT_SECRET = 'LVYCDQL31ZGW3JHXLMURUPIXUTEIGSSWIUG3OJ1QUH11HHTS' # your Foursquare Secret

VERSION = '20180605' # Foursquare API version

print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:


## Exploration for SEATTLE
## Using Foursquare API for getting 100 (max) venues nearby seattle in 500 mtr radius.

In [9]:
# type your answer here
latitude=seattle_lat_lang[0]
longitude=seattle_lat_lang[1]
VERSION=20180604
radius=500
LIMIT=100

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print(url)
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

import requests
results = requests.get(url).json()


https://api.foursquare.com/v2/venues/explore?client_id=WI10VXATDOABDVBGEVP5NS2ONF5SRWCTIJXRFP44XWHUY1MD&client_secret=LVYCDQL31ZGW3JHXLMURUPIXUTEIGSSWIUG3OJ1QUH11HHTS&ll=47.6062,-122.3321&v=20180604&radius=500&limit=100
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


#### Normalizing and creating a dataframe out of foursquare API response

In [10]:
# function that extracts the category of the venue
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Columbia Tower Club,Social Club,47.604507,-122.330484
1,Tulio Ristorante,Italian Restaurant,47.607460,-122.332410
2,Metropolitan Grill,Steakhouse,47.604617,-122.334280
3,Juicy Cafe,Café,47.604329,-122.330958
4,Red Bowls,Noodle House,47.604654,-122.333211


In [11]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

85 venues were returned by Foursquare.


### creating one-hot encoded df on the basis of categories of venue.

In [12]:
# one hot encoding
seattle_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# # add neighborhood column back to dataframe
seattle_onehot['venue'] = nearby_venues['name'] 

# # move neighborhood column to the first column
fixed_columns = [seattle_onehot.columns[-1]] + list(seattle_onehot.columns[:-1])
seattle_onehot = seattle_onehot[fixed_columns]

seattle_onehot.head()

,venue,American Restaurant,Art Museum,Asian Restaurant,BBQ Joint,Breakfast Spot,Building,Burger Joint,Café,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Dessert Shop,Donut Shop,Eastern European Restaurant,Gift Shop,Gym,Halal Restaurant,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Lounge,Music Venue,New American Restaurant,Noodle House,Office,Outdoor Supply Store,Park,Pizza Place,Poke Place,Pub,Public Art,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Social Club,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Thai Restaurant,Theater,Tour Provider,Wine Bar
0,Columbia Tower Club,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Tulio Ristorante,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Metropolitan Grill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,Juicy Cafe,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Red Bowls,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
seattle_final=seattle_onehot.copy()

In [14]:
print(seattle_onehot.shape)
seattle_onehot.drop('venue', axis=1).head(2)

(85, 50)


,American Restaurant,Art Museum,Asian Restaurant,BBQ Joint,Breakfast Spot,Building,Burger Joint,Café,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Dessert Shop,Donut Shop,Eastern European Restaurant,Gift Shop,Gym,Halal Restaurant,Historic Site,Hotel,Hotel Bar,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Lounge,Music Venue,New American Restaurant,Noodle House,Office,Outdoor Supply Store,Park,Pizza Place,Poke Place,Pub,Public Art,Salad Place,Sandwich Place,Scenic Lookout,Seafood Restaurant,Social Club,Spa,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Thai Restaurant,Theater,Tour Provider,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Using K-means CLustering venues on the basis of venue category for analysing venues.

In [15]:
kclusters = 5

seattle_onehot = seattle_onehot.drop('venue', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(seattle_onehot)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 0, 2, 0, 0, 0, 1], dtype=int32)

#### assinging labels to venues

In [16]:
#seattle_onehot['Cluster_Labels'] = kmeans.labels_
nearby_venues['Cluster_Labels'] = kmeans.labels_    #      .head()


In [17]:
nearby_venues[nearby_venues['Cluster_Labels']==1].head()

,name,categories,lat,lng,Cluster_Labels
9,Fairmont Olympic Hotel,Hotel,47.608009,-122.333984,1
14,W Seattle,Hotel,47.607404,-122.333668,1
20,"Loews Hotel 1000, Seattle",Hotel,47.605051,-122.336105,1
28,Renaissance Seattle Hotel,Hotel,47.606667,-122.330685,1
36,Kimpton Alexis Hotel Seattle,Hotel,47.604880,-122.336900,1


In [18]:
# nearby_venues.drop('Cluster Labels', inplace=True, axis=1)

In [19]:
nearby_venues.groupby(by=['Cluster_Labels','categories']).count().reset_index().sort_values(['Cluster_Labels','name'],ascending=False).head(12)
# type(nearby_venues.groupby(by=['Cluster_Labels','categories']).count().reset_index())

,Cluster_Labels,categories,name,lat,lng
48,4,Italian Restaurant,4,4,4
47,3,Coffee Shop,8,8,8
46,2,New American Restaurant,4,4,4
45,1,Hotel,8,8,8
7,0,Café,5,5,5
32,0,Salad Place,3,3,3
33,0,Sandwich Place,3,3,3
0,0,American Restaurant,2,2,2
10,0,Concert Hall,2,2,2
12,0,Donut Shop,2,2,2


#### INFERENCE :-

1. As per our belief ( mentioned in the background part of this project on github - https://github.com/lalitsomnathe/Coursera_Capstone/blob/master/Nearby_Seattle_food_choice_exploration_lalitsomnathe_Intro.txt ) we can definitely see large veriety of food categories namely Italian, New American, Asian etc. in radius of just 500 mtrs.

2. Also, as seattle being mostly commercial place we see most number of cafe+coffee shops = 14 , the maximum. Coffee shops are great places for meetups and catching up for short business talks.

3. Similarly, for employee occomodation for short visits there are good number of hotels (7) nearby.

4. We can see most of the venues are eateries, stores for home appliances,clothing stores are not there in the list. ( as it is mostly a commercial place ).

#### Visualizing all the venues imposed with corressponding cluster colour

In [34]:
# create map
seattle_map_clusters = folium.Map(location=[seattle_lat_lang[0], seattle_lat_lang[1]], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, long, poi, cluster in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(seattle_map_clusters)
       
seattle_map_clusters

In just 50 blocks of perimeter we see lot of varieties of foods right from very generic one like salad to all the way upto specific country food like Italian, asian etc.

#### EXploring only for asian food 

In [21]:
nearby_venues.head()
seattle_nearby_aisianfood=nearby_venues[nearby_venues['categories']=='Asian Restaurant']
seattle_nearby_aisianfood

,name,categories,lat,lng,Cluster_Labels
60,Wild Ginger,Asian Restaurant,47.608891,-122.337344,0


In [22]:
# add markers to map

seattle_map_venues=folium.Map(location=[seattle_lat_lang[0], seattle_lat_lang[1]], zoom_start=15)

# for lat, lng, label in zip(seattle_lat_lang[0], seattle_lat_lang[1], seattle_lat_lang[2]):
# for i in (seattle_nearby_aisianfood[lat],seattle_nearby_aisianfood[lng],seattle_nearby_aisianfood[name]):
for i in seattle_nearby_aisianfood.values:
    lat, lng, label=i[2],i[3],i[1]
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(seattle_map_venues)  
    
seattle_map_venues

#### INFERENCE :-
So for asian food lovers there are three restuarants in 50 blocks perimeter. For small 500 mtr range 3 restaurants is a good number to have a try.

In [23]:
#print (bellevue_lat_lang)
# print (tacoma_lat_lang)
# print (everett_lat_lang)

#### I WILL BE DOING SIMILAR ANALYSIS FOR OTHER METRO DIVISIONS.
So most of the codes written above will be same .

## Exploration for bellevue
## Using Foursquare API for getting 500 (max) venues nearby seattle in 5000 mtr radius.

In [24]:
# USING FOURSQUARE API for getting nearby venues details.

latitude=bellevue_lat_lang[0]
longitude=bellevue_lat_lang[1]
VERSION=20180604
# radius=500
# LIMIT=100
radius=5000
LIMIT=500



url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print(url)
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

#import requests
results = requests.get(url).json()

#################################################################

# FUNCTION THAT EXTRACTS THE CATEGORY OF THE VENUE

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
bellevue_nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
bellevue_nearby_venues =bellevue_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
bellevue_nearby_venues['venue.categories'] = bellevue_nearby_venues.apply(get_category_type, axis=1)
# clean columns
bellevue_nearby_venues.columns = [col.split(".")[-1] for col in bellevue_nearby_venues.columns]

print('-----------------------------------------------------------')
print('{} venues were returned by Foursquare.'.format(bellevue_nearby_venues.shape[0]))
print('-----------------------------------------------------------')

# bellevue_nearby_venues.head()

#################################################################

# ONE HOT ENCODING FOR VENUES ON THE BASIS OF CATEGORIES

bellevue_onehot = pd.get_dummies(bellevue_nearby_venues[['categories']], prefix="", prefix_sep="")
# # add neighborhood column back to dataframe
bellevue_onehot['venue'] = bellevue_nearby_venues['name'] 
# # move neighborhood column to the first column
fixed_columns = [bellevue_onehot.columns[-1]] + list(bellevue_onehot.columns[:-1])
bellevue_onehot = bellevue_onehot[fixed_columns]
# bellevue_onehot.head()

#################################################################

# USING K-means CLUSTERRING 

kclusters = 5
bellevue_onehot = bellevue_onehot.drop('venue', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bellevue_onehot)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# ASSIGN cluster labels to venues
bellevue_nearby_venues['Cluster_Labels'] = kmeans.labels_    

##################################################################

# create map
bellevue_map_clusters = folium.Map(location=[bellevue_lat_lang[0], bellevue_lat_lang[1]], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, long, poi, cluster in zip(bellevue_nearby_venues['lat'], bellevue_nearby_venues['lng'], bellevue_nearby_venues['name'], bellevue_nearby_venues['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(bellevue_map_clusters)
       
bellevue_map_clusters

https://api.foursquare.com/v2/venues/explore?client_id=WI10VXATDOABDVBGEVP5NS2ONF5SRWCTIJXRFP44XWHUY1MD&client_secret=LVYCDQL31ZGW3JHXLMURUPIXUTEIGSSWIUG3OJ1QUH11HHTS&ll=47.6104,-122.2007&v=20180604&radius=5000&limit=500
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-----------------------------------------------------------
100 venues were returned by Foursquare.
-----------------------------------------------------------


In [25]:
bellevue_onehot.head()

,American Restaurant,Asian Restaurant,Bagel Shop,Bakery,Bar,Beach,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Café,Cheese Shop,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Cosmetics Shop,Cycle Studio,Department Store,Dessert Shop,Donut Shop,Dumpling Restaurant,Electronics Store,Furniture / Home Store,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Italian Restaurant,Japanese Restaurant,Lingerie Store,Liquor Store,Marijuana Dispensary,Mediterranean Restaurant,Mexican Restaurant,Movie Theater,Museum,New American Restaurant,Park,Pizza Place,Poke Place,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shopping Mall,Spa,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Toy / Game Store,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
bellevue_nearby_venues.head()

,name,categories,lat,lng,Cluster_Labels
0,Bellevue Downtown Park,Park,47.612860,-122.203805,0
1,Pagliacci Pizza,Pizza Place,47.610483,-122.206542,0
2,Araya's Place,Vegetarian / Vegan Restaurant,47.610242,-122.203017,0
3,H-Mart,Grocery Store,47.611017,-122.195853,0
4,Broiler Bay Burgers,Burger Joint,47.610515,-122.197616,0


In [27]:
bellevue_nearby_venues.groupby(by=['Cluster_Labels','categories']).count().reset_index().sort_values(['Cluster_Labels','name'],ascending=False).head(15)

,Cluster_Labels,categories,name,lat,lng
62,4,Italian Restaurant,4,4,4
61,3,Spa,5,5,5
60,2,Toy / Game Store,3,3,3
59,1,Coffee Shop,6,6,6
27,0,Grocery Store,4,4,4
52,0,Steakhouse,4,4,4
48,0,Shopping Mall,3,3,3
3,0,Bakery,2,2,2
4,0,Bar,2,2,2
5,0,Beach,2,2,2


#### INFERENCE :-

1. For bellevue division, there are good number of coffee shops but almost half as compared to seattle division.

2. We can see venues like "Clothing Store", "Furniture / Home Store", "Bakery" etc. which were not in the list for seattle division.
   These venues clearly suggest a good housing population as compared to seattle.( as furniture store, clothing stores, bakery are for house hold goods).

3. I can see for this division as well, there are some Italian restaurants nearby just like seattle.

4. We also see SteakHouse, Shopping Mall, Toy/Game Store, Bookstore which indicates a good number of population residing in this area just like point 2 above when compared to seattle.


## Exploration for TACOMA
## Using Foursquare API for getting 500 (max) venues nearby seattle in 5000 mtr radius.

In [28]:
# type your answer here
latitude=tacoma_lat_lang[0]
longitude=tacoma_lat_lang[1]
VERSION=20180604
# radius=500
# LIMIT=100
radius=5000
LIMIT=500


url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print(url)
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

#import requests
results = requests.get(url).json()

#################################################################
# function that extracts the category of the venue
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
tacoma_nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
tacoma_nearby_venues =tacoma_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
tacoma_nearby_venues['venue.categories'] = tacoma_nearby_venues.apply(get_category_type, axis=1)
# clean columns
tacoma_nearby_venues.columns = [col.split(".")[-1] for col in tacoma_nearby_venues.columns]

print('-----------------------------------------------------------')
print('{} venues were returned by Foursquare.'.format(tacoma_nearby_venues.shape[0]))
print('-----------------------------------------------------------')

# tacoma_nearby_venues.head()

#################################################################

# one hot encoding
tacoma_onehot = pd.get_dummies(tacoma_nearby_venues[['categories']], prefix="", prefix_sep="")
# # add neighborhood column back to dataframe
tacoma_onehot['venue'] = tacoma_nearby_venues['name'] 
# # move neighborhood column to the first column
fixed_columns = [tacoma_onehot.columns[-1]] + list(tacoma_onehot.columns[:-1])
tacoma_onehot = tacoma_onehot[fixed_columns]
# tacoma_onehot.head()

#################################################################
kclusters = 5
tacoma_onehot = tacoma_onehot.drop('venue', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tacoma_onehot)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

tacoma_nearby_venues['Cluster_Labels'] = kmeans.labels_    
##################################################################

# create map
tacoma_map_clusters = folium.Map(location=[tacoma_lat_lang[0], tacoma_lat_lang[1]], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, long, poi, cluster in zip(tacoma_nearby_venues['lat'], tacoma_nearby_venues['lng'], tacoma_nearby_venues['name'], tacoma_nearby_venues['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(tacoma_map_clusters)
       
tacoma_map_clusters

https://api.foursquare.com/v2/venues/explore?client_id=WI10VXATDOABDVBGEVP5NS2ONF5SRWCTIJXRFP44XWHUY1MD&client_secret=LVYCDQL31ZGW3JHXLMURUPIXUTEIGSSWIUG3OJ1QUH11HHTS&ll=47.2529,-122.4443&v=20180604&radius=5000&limit=500
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-----------------------------------------------------------
100 venues were returned by Foursquare.
-----------------------------------------------------------


In [29]:
tacoma_nearby_venues.head()

,name,categories,lat,lng,Cluster_Labels
0,Tacoma Comedy Club,Comedy Club,47.253953,-122.441358,0
1,Grand Cinema,Indie Movie Theater,47.257894,-122.444562,0
2,Simply Panini,Sandwich Place,47.251838,-122.439931,0
3,Dorky's Arcade,Arcade,47.255616,-122.439196,0
4,YMCA,Gym,47.249849,-122.441027,0


In [30]:
tacoma_nearby_venues.groupby(by=['Cluster_Labels','categories']).count().reset_index().sort_values(['Cluster_Labels','name'],ascending=False).head(15)


,Cluster_Labels,categories,name,lat,lng
63,4,Bar,4,4,4
62,3,Pizza Place,4,4,4
61,2,American Restaurant,5,5,5
60,1,Brewery,5,5,5
14,0,Coffee Shop,4,4,4
45,0,Pub,4,4,4
48,0,Sandwich Place,4,4,4
12,0,Café,3,3,3
40,0,Museum,3,3,3
0,0,Antique Shop,2,2,2


#### INFERENCE :-

1. For tacoma division, we see most of american restaurant. This clearly matches with our reference (http://worldpopulationreview.com/us-cities/tacoma-population/) which says tacoma's almost 65% population is white.

2. The link also says, Tacoma hosts around 8% of asian people. This also matches as we found 2 Asian Restaurants in the vicinity.

3. There are some Museums (3+2) , and Antique shops (2) suggests good place for the people to spend some calm quality time. This is also mentioned in wikipage :- https://en.wikipedia.org/wiki/Tacoma,_Washington referencing Tacoma's Museum District.
So, for history and art lovers Tacoma is good place to spare some time.

4. Presence of Grocery Store, Brewery, pub etc. suggests a housing population residing nearby.

5. Also, many venues are segregated nearby coastline. As we move further away from coastline, number of venues are decreasing.

## Exploration for EVERETT
## Using Foursquare API for getting 500 (max) venues nearby seattle in 5000 mtr radius.

In [31]:
# USING FOURSQUARE API for getting nearby venues details.

latitude=everett_lat_lang[0]
longitude=everett_lat_lang[1]
VERSION=20180604
# radius=500
# LIMIT=100
radius=5000
LIMIT=500


url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print(url)
print('+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

#import requests
results = requests.get(url).json()

#################################################################

# FUNCTION THAT EXTRACTS THE CATEGORY OF THE VENUE

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

venues = results['response']['groups'][0]['items']
    
everett_nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
everett_nearby_venues =everett_nearby_venues.loc[:, filtered_columns]
# filter the category for each row
everett_nearby_venues['venue.categories'] = everett_nearby_venues.apply(get_category_type, axis=1)
# clean columns
everett_nearby_venues.columns = [col.split(".")[-1] for col in everett_nearby_venues.columns]

print('-----------------------------------------------------------')
print('{} venues were returned by Foursquare.'.format(everett_nearby_venues.shape[0]))
print('-----------------------------------------------------------')

# everett_nearby_venues.head()

#################################################################

# ONE HOT ENCODING FOR VENUES ON THE BASIS OF CATEGORIES

everett_onehot = pd.get_dummies(everett_nearby_venues[['categories']], prefix="", prefix_sep="")
# # add neighborhood column back to dataframe
everett_onehot['venue'] = everett_nearby_venues['name'] 
# # move neighborhood column to the first column
fixed_columns = [everett_onehot.columns[-1]] + list(everett_onehot.columns[:-1])
everett_onehot = everett_onehot[fixed_columns]
# everett_onehot.head()

#################################################################

# USING K-means CLUSTERRING 

kclusters = 5
everett_onehot = everett_onehot.drop('venue', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(everett_onehot)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# ASSIGN cluster labels to venues
everett_nearby_venues['Cluster_Labels'] = kmeans.labels_    

##################################################################

# create map
everett_map_clusters = folium.Map(location=[everett_lat_lang[0], everett_lat_lang[1]], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, long, poi, cluster in zip(everett_nearby_venues['lat'], everett_nearby_venues['lng'], everett_nearby_venues['name'], everett_nearby_venues['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(everett_map_clusters)
       
everett_map_clusters

https://api.foursquare.com/v2/venues/explore?client_id=WI10VXATDOABDVBGEVP5NS2ONF5SRWCTIJXRFP44XWHUY1MD&client_secret=LVYCDQL31ZGW3JHXLMURUPIXUTEIGSSWIUG3OJ1QUH11HHTS&ll=47.979,-122.2021&v=20180604&radius=5000&limit=500
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
-----------------------------------------------------------
100 venues were returned by Foursquare.
-----------------------------------------------------------


In [32]:
everett_nearby_venues.head()

,name,categories,lat,lng,Cluster_Labels
0,FunkoHQ,Toy / Game Store,47.980455,-122.207359,0
1,Narrative Coffee Co.,Coffee Shop,47.978376,-122.207051,4
2,Brooklyn Brothers Pizza,Pizza Place,47.979342,-122.203162,1
3,Terracotta Red,Asian Restaurant,47.978977,-122.192648,0
4,Xfinity Arena,Hockey Arena,47.978837,-122.202907,0


In [33]:
everett_nearby_venues.groupby(by=['Cluster_Labels','categories']).count().reset_index().sort_values(['Cluster_Labels','name'],ascending=False).head(15)

,Cluster_Labels,categories,name,lat,lng
65,4,Coffee Shop,6,6,6
64,3,Mexican Restaurant,3,3,3
63,2,Sandwich Place,4,4,4
62,1,Pizza Place,5,5,5
5,0,Bar,3,3,3
27,0,Grocery Store,3,3,3
35,0,Park,3,3,3
2,0,Asian Restaurant,2,2,2
11,0,Breakfast Spot,2,2,2
12,0,Brewery,2,2,2


1. So this is first time we are seeing Mexican restaurants in the top. This suggest a good mexican population surrounding everett.
I check online and found https://www.quora.com/Where-do-the-most-Mexicans-live-in-Seattle which says around 7.7% of population in everett are mexicans.
So the data is matching . Hurray!.

2. Also we see one Indian Restaurant in 500mtrs radius in everett.

#### INFERENCE :-

1. Little bit like bellevue division, everett shows good number of coffee shops but almost half as compared to seattle division.

2. Grocery store, Park, Bar, Brewery, Gym etc. points that Everett is also a household place just like Bellevue.

3. Good options for asian, chinese and mexican food lovers in this division. (unlike seattle for italian food).

#### So for household purpose I would say, Bellevue and Everett metro divisions are on top as they suggests good grocery/eateriy venues.

#### For asian food lovers, TACOMA and EVERETT divisions are must to visit.

#### In almost every division, there are venues for Italian food. SO for italian food lovers, not to worry about location :)

#### Inspite of being a good metro-politan, in Bellevue there are very less aisan food choices. So if someone want to give a try to open an asian restaurant, he/she could setup in BEllevue.

THANK YOU :)